In [1]:
! pip install statsmodels

You should consider upgrading via the 'c:\users\saidj\onedrive\documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\scripts\python.exe -m pip install --upgrade pip' command.


# Libraries

In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('pruebillas.ipynb'), '..'))
os.chdir(root_dir)
sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps, resample_apply, barssince
import pandas as pd
import talib as ta
import numpy as np

from backbone.utils.general_purpose import diff_pips

# Data

In [24]:
symbols_path = './backbone/data/backtest/symbols/USDCAD.csv'
df = pd.read_csv(symbols_path)
df['Date'] = pd.to_datetime(df.Date)
df = df.set_index('Date')
df

,Open,High,Low,Close,Volume,direction,Group,consecutive_candles
Date,,,,,,,,
2021-01-01 00:00:00,1.27316,1.27316,1.27316,1.27316,0.000000e+00,-1,1,1
2021-01-01 00:01:00,1.27316,1.27316,1.27316,1.27316,0.000000e+00,-1,1,2
2021-01-01 00:02:00,1.27316,1.27316,1.27316,1.27316,0.000000e+00,-1,1,3
2021-01-01 00:03:00,1.27316,1.27316,1.27316,1.27316,0.000000e+00,-1,1,4
2021-01-01 00:04:00,1.27316,1.27316,1.27316,1.27316,0.000000e+00,-1,1,5
...,...,...,...,...,...,...,...,...
2024-01-01 23:55:00,1.32475,1.32475,1.32473,1.32473,2.400000e+06,-1,553947,1
2024-01-01 23:56:00,1.32474,1.32479,1.32455,1.32455,3.360000e+07,-1,553947,2
2024-01-01 23:57:00,1.32456,1.32463,1.32447,1.32447,2.965000e+07,-1,553947,3


In [25]:
import numpy as np
import pandas as pd


def consecutive(close, open, n):
    # Determina la dirección de las velas: 1 para alcista, -1 para bajista, 0 para sin cambio
    direction = np.where(close > open, 1, np.where(close < open, -1, 0))
    
    # Detecta cambios en la dirección
    changes = np.diff(direction, prepend=direction[0])
    group = np.cumsum(changes != 0)
    
    # Contar el número de velas consecutivas dentro de cada grupo
    consecutive_candles = np.zeros_like(group)
    unique_groups, counts = np.unique(group, return_counts=True)
    for grp, count in zip(unique_groups, counts):
        consecutive_candles[group == grp] = np.arange(1, count + 1)
    
    # Crear una serie con valores 0 por defecto
    result = np.zeros_like(direction)
    
    # Asignar 1 o -1 para secuencias que tienen exactamente n velas consecutivas
    result[consecutive_candles == n] = direction[consecutive_candles == n]
    
    return list(result)



In [26]:
def  optim_func(series):
    return series['Return [%]'] / (series['Max. Trade Duration'])

# Strategy

In [27]:
from backtesting import Strategy

class ConsecutiveCandlesGrid(Strategy):
    n = 7
    percent_profit = 0.001
    percent_loss = 0.05
    pip_value = 0.0001
    grid_size = 15
    position_size = 3000
    rsi_down_threshold = 30
    rsi_up_threshold = 70
    max_minutes_in_trade = 400

    def init(self):
        self.consecutive = self.I(consecutive, self.data.Close, self.data.Open, n=self.n)
        self.rsi = self.I(ta.RSI, self.data.Close, 14)

    
    def next(self):
        close_prices = self.data.Close
        open_prices = self.data.Open
        actual_close_price = close_prices[-1]

        today = self.data.index[-1]
        today = today.tz_localize('UTC').tz_convert('UTC')  # Si el timestamp ya está en UTC pero no tiene la info

        # Verificar si hay posiciones abiertas
        if self.position:
            last_trade = self.trades[-1]
            first_trade = self.trades[0]

            profit_target = self.equity * self.percent_profit
            loss_target = self.equity * self.percent_loss

            total_profit = sum(trade.pl for trade in self.trades)
            
            last_trade_pl = last_trade.pl
            pips = diff_pips(last_trade.entry_price, actual_close_price, pip_value=self.pip_value, absolute=True)

            dinamic_grid_size = self.grid_size * (len(self.trades))

            time_in_position = (today - first_trade.entry_time.tz_localize('UTC').tz_convert('UTC'))
            time_in_position = time_in_position.total_seconds() // 60
            
            if time_in_position >= self.max_minutes_in_trade and total_profit >=0:
                self.position.close()

            elif total_profit <= -loss_target or total_profit >= profit_target:
                self.position.close()

            # Añadir posición en caso de que el profit/loss entre dentro del rango
            elif self.position.is_long and last_trade_pl < 0 and pips > dinamic_grid_size:
                self.buy(size=self.position_size)
                    
            elif self.position.is_short and last_trade_pl < 0 and pips > dinamic_grid_size:
                self.sell(size=self.position_size)

        else:

            # Abrir posición en función de las velas consecutivas 
            
            weekday = today.weekday()

            if self.consecutive == -1 and self.rsi > self.rsi_down_threshold and self.rsi < self.rsi_up_threshold and weekday < 4:
                self.buy(size=self.position_size)
                
            elif self.consecutive == 1 and self.rsi > self.rsi_down_threshold and self.rsi < self.rsi_up_threshold and weekday < 4:
                self.sell(size=self.position_size)


# WFO

In [30]:
def walk_forward(
        strategy,
        data_full,
        warmup_bars,
        lookback_bars=28*1440,
        validation_bars=7*1440,
        cash=10_000_000, 
        commission=0.002,
        margin=1/30):

    stats_master = []

    for i in range(lookback_bars, len(data_full)-validation_bars, validation_bars):

        print(i)

        # To do anchored walk-forward, just set the first slice here to 0
        train_data = data_full.iloc[i-lookback_bars: i]

        bt_training = Backtest(train_data, strategy, cash=cash, commission=commission, margin=margin)
        stats_training = bt_training.optimize(
                n=[6, 7, 8, 9],
                percent_profit=[0.0005, 0.0008],
                percent_loss=[0.01, 0.02, 0.03],
                grid_size=[10, 20, 25, 30],
                rsi_up_threshold=[55, 60, 70],
                rsi_down_threshold=[30, 40, 45],
                maximize='Return [%]',
                max_tries=100,
        )
        
        validation_data = data_full.iloc[i-warmup_bars:i+validation_bars]
        bt_validation = Backtest(validation_data, strategy, cash=cash, commission=commission)
        stats_validation = bt_validation.run(
                n=stats_training._strategy.n,
                percent_profit=stats_training._strategy.percent_profit,
                percent_loss=stats_training._strategy.percent_loss,
                grid_size=stats_training._strategy.grid_size,
                rsi_up_threshold=stats_training._strategy.rsi_up_threshold,
                rsi_down_threshold=stats_training._strategy.rsi_down_threshold,
                position_size=stats_training._strategy.position_size,
        )

        stats_master.append(stats_validation)

    return stats_master

In [31]:
import pickle


lookback_bars = 28*1440
validation_bars = 7*1440
warmup_bars = 14*60

if os.path.exists("stats.pickle"):
    with open("stats.pickle", "rb") as f:
        stats = pickle.load(f)
else:
    stats = walk_forward(ConsecutiveCandlesGrid, df, warmup_bars = warmup_bars)
    with open("stats.pickle", "wb") as f:
        pickle.dump(stats, f)


40320


  0%|          | 0/9 [00:00<?, ?it/s]

50400


  0%|          | 0/9 [00:00<?, ?it/s]

60480


...

1209600


  0%|          | 0/9 [00:00<?, ?it/s]

1219680


  0%|          | 0/9 [00:00<?, ?it/s]

MemoryError: 